In [30]:
from dash import Dash, html, dcc, callback, Output, Input, State
import plotly.express as px
import pandas as pd
import numpy as np
import sqlite3

from dash import Dash, html, dcc, dash_table, Input, Output, State
import pandas as pd
from datetime import datetime

### SQL Request

In [48]:
conn = sqlite3.connect('../data/database.db')
conn_memory = sqlite3.connect(':memory:')

start_date = '2024-01'
end_date = '2024-02'

query = f'SELECT * \
        FROM NODES_CONGESTION_MONTHLY \
        WHERE Date >= "{start_date}" AND Date <= "{end_date}" \
        ORDER BY Congestion DESC;'
df = pd.read_sql_query(query, conn)
df.to_sql('NODES_CONGESTION_DAILY', conn_memory, if_exists='replace', index=False)
display(df)

limit = 100
query = f'SELECT DISTINCT \
            a.Date as Date, \
            a.Node as Node1, \
            b.Node as Node2, \
            a.Congestion - b.Congestion as Congestion \
        FROM NODES_CONGESTION_DAILY a \
        CROSS JOIN NODES_CONGESTION_DAILY b \
        WHERE a.Node < b.Node OR a.Node > b.Node \
        ORDER BY Congestion \
        LIMIT {limit};'
df = pd.read_sql_query(query, conn_memory)
df.to_sql('NODES_PAIR_CONGESTION', conn_memory, if_exists='replace', index=False)

display(df)

,Date,Node,Congestion
0,2024-01-01 00:00:00,WAUE.BEPM.LCS1,101738.8826
1,2024-01-01 00:00:00,WAUE.BEPM.LCS2,101738.8826
2,2024-01-01 00:00:00,WAUE.BEPM.LCS3,101738.8826
3,2024-01-01 00:00:00,WAUE.VOLT.0172,65874.5078
4,2024-01-01 00:00:00,WAUE.BEPM.LCS4,64297.7439
...,...,...,...
1195,2024-01-01 00:00:00,SPS.NICHOLS2,-18027.7871
1196,2024-01-01 00:00:00,SPS.SPINSPUR,-20110.1634
1197,2024-01-01 00:00:00,MPS.AMUE.OUTLAW,-27231.2044
1198,2024-01-01 00:00:00,MPS.ROCKCREEK,-27231.2044


,Date,Node1,Node2,Congestion
0,2024-01-01 00:00:00,MPS.OSBORN,WAUE.BEPM.LCS1,-137217.6062
1,2024-01-01 00:00:00,MPS.OSBORN,WAUE.BEPM.LCS2,-137217.6062
2,2024-01-01 00:00:00,MPS.OSBORN,WAUE.BEPM.LCS3,-137217.6062
3,2024-01-01 00:00:00,MPS.AMUE.OUTLAW,WAUE.BEPM.LCS1,-128970.0870
4,2024-01-01 00:00:00,MPS.AMUE.OUTLAW,WAUE.BEPM.LCS2,-128970.0870
...,...,...,...,...
95,2024-01-01 00:00:00,SPS.ENGCARBON1,WAUE.BEPM.LCS3,-117988.4007
96,2024-01-01 00:00:00,SPS.RICHRDSN1,WAUE.BEPM.LCS1,-117988.4007
97,2024-01-01 00:00:00,SPS.RICHRDSN1,WAUE.BEPM.LCS2,-117988.4007
98,2024-01-01 00:00:00,SPS.RICHRDSN1,WAUE.BEPM.LCS3,-117988.4007


In [43]:
node_a = 'MPS.OSBORN'
node_b = 'WAUE.BEPM.LRSE'

node_a = 'WAUE.BEPM.LCS1'
node_b = 'MPS.OSBORN'

query = f'SELECT * \
        FROM NODES_PAIR_CONGESTION \
        WHERE Node1 == "{node_a}" AND Node2 == "{node_b}"'
df = pd.read_sql_query(query, conn_memory)
display(df)

,Date,Node1,Node2,Congestion
0,2024-01-01 00:00:00,WAUE.BEPM.LCS1,MPS.OSBORN,137217.6062


### Dashboard elements

In [ ]:
app = Dash(__name__)

app.layout = html.Div([
    html.H1("Dashboard pour les données de congestion des noeuds"),
    html.Div([
        html.Label("Date de début:"),
        dcc.DatePickerSingle(
            id='date-debut',
            # min_date_allowed=df['Date'].min(),
            # max_date_allowed=df['Date'].max(),
            # initial_visible_month=df['Date'].min(),
            # date=df['Date'].min(),
            style={'marginRight': '20px'}
        ),
        html.Label("Date de fin:"),
        dcc.DatePickerSingle(
            id='date-fin',
            # min_date_allowed=df['Date'].min(),
            # max_date_allowed=df['Date'].max(),
            # initial_visible_month=df['Date'].max(),
            # date=df['Date'].max(),
            style={'marginRight': '20px'}
        ),
        html.Label("Nombre maximum de lignes:"),
        dcc.Input(
            id='max-rows',
            type='number',
            value=20,
            min=1,
            max=len(df),
            style={'marginLeft': '10px', 'width': '100px'}
        ),
        html.Button('Appliquer les filtres', id='apply-button', 
                   style={'backgroundColor': '#4CAF50',
                          'color': 'white',
                          'padding': '10px 20px',
                          'border': 'none',
                          'borderRadius': '5px',
                          'cursor': 'pointer'})
    ], style={'marginBottom': '20px'}),

    # Tableau
    dash_table.DataTable(
        id='table',
        columns=[
            {'name': col, 'id': col} for col in df.columns
        ],
        data=df.to_dict('records'),
        page_size=10,
        style_table={'overflowX': 'auto'},
        style_cell={'textAlign': 'left', 'padding': '10px'},
        style_header={
            'backgroundColor': 'rgb(230, 230, 230)',
            'fontWeight': 'bold'
        },
        sort_action='native',  # Permet le tri
        filter_action='native'  # Permet le filtrage
    )
])

@app.callback(
    Output('table', 'data'),
    Output('table', 'page_size'),
    [Input('apply-button', 'n_clicks')],
    [State('date-debut', 'date'),
     State('date-fin', 'date'),
     State('max-rows', 'value')]
)
def update_table(n_clicks, date_debut, date_fin, max_rows):
    # Convertir les dates en datetime
    date_debut = pd.to_datetime(date_debut)
    date_fin = pd.to_datetime(date_fin)
    
    # Filtrer les données
    filtered_df = df[
        (df['Date'] >= date_debut) &
        (df['Date'] <= date_fin)
    ].sort_values('Congestion', ascending=False)
    
    # Limiter le nombre de lignes
    filtered_df = filtered_df.head(max_rows)

    return filtered_df.to_dict('records'), max_rows

if __name__ == '__main__':
    app.run(debug=True, port=5050)
    

In [ ]:


app = Dash(__name__)

# Lire les données initiales
df = pd.read_sql_query('SELECT * FROM NODES_CONGESTION_DAILY ORDER BY Congestion DESC', conn)
df['Date'] = pd.to_datetime(df['Date'])  # Assurez-vous que la colonne date est au bon format

app.layout = html.Div([
    html.H1("Dashboard pour les données de congestion des noeuds"),
    
    # Conteneur pour les contrôles
    html.Div([
        # Sélecteur de dates
        html.Div([
            html.Label("Date de début:"),
            dcc.DatePickerSingle(
                id='date-debut',
                min_date_allowed=df['Date'].min(),
                max_date_allowed=df['Date'].max(),
                initial_visible_month=df['Date'].min(),
                date=df['Date'].min(),
                style={'marginRight': '20px'}
            ),
            
            html.Label("Date de fin:"),
            dcc.DatePickerSingle(
                id='date-fin',
                min_date_allowed=df['Date'].min(),
                max_date_allowed=df['Date'].max(),
                initial_visible_month=df['Date'].max(),
                date=df['Date'].max(),
                style={'marginRight': '20px'}
            ),
            html.Label("Nombre maximum de lignes:"),
            dcc.Input(
                id='max-rows',
                type='number',
                value=10,
                min=1,
                max=len(df),
                style={'marginLeft': '10px', 'width': '100px'}
            ),
        ], style={'marginBottom': '20px'}),
        
        # Bouton pour appliquer les filtres
        html.Button('Appliquer les filtres', id='apply-button', 
                   style={'backgroundColor': '#4CAF50',
                          'color': 'white',
                          'padding': '10px 20px',
                          'border': 'none',
                          'borderRadius': '5px',
                          'cursor': 'pointer'})
    ], style={'padding': '20px', 'backgroundColor': '#f8f9fa', 'borderRadius': '5px'}),
    
    # Tableau
    dash_table.DataTable(
        id='table',
        columns=[
            {'name': col, 'id': col} for col in df.columns
        ],
        data=df.to_dict('records'),
        page_size=10,
        style_table={'overflowX': 'auto'},
        style_cell={'textAlign': 'left', 'padding': '10px'},
        style_header={
            'backgroundColor': 'rgb(230, 230, 230)',
            'fontWeight': 'bold'
        },
        sort_action='native',  # Permet le tri
        filter_action='native'  # Permet le filtrage
    )
])

@app.callback(
    Output('table', 'data'),
    Output('table', 'page_size'),
    [Input('apply-button', 'n_clicks')],
    [State('date-debut', 'date'),
     State('date-fin', 'date'),
     State('max-rows', 'value')]
)
def update_table(n_clicks, date_debut, date_fin, max_rows):
    # Convertir les dates en datetime
    date_debut = pd.to_datetime(date_debut)
    date_fin = pd.to_datetime(date_fin)
    
    # Filtrer les données
    filtered_df = df[
        (df['Date'] >= date_debut) &
        (df['Date'] <= date_fin)
    ].sort_values('Congestion', ascending=False)
    
    # Limiter le nombre de lignes
    filtered_df = filtered_df.head(max_rows)
    
    return filtered_df.to_dict('records'), max_rows

if __name__ == '__main__':
    app.run(debug=True, port=5050)

In [45]:
from dash import Dash, html, dcc, Input, Output
import plotly.express as px
import pandas as pd
import sqlite3

# Initialiser l'application
app = Dash(__name__)

conn = sqlite3.connect('../data/database.db')
# Lire les données
df = pd.read_sql_query('SELECT * FROM NODES_CONGESTION_DAILY ORDER BY Congestion DESC', conn)
df['Date'] = pd.to_datetime(df['Date'])

app.layout = html.Div([
    html.H1("Analyse des Congestions", style={'textAlign': 'center'}),
    
    # Contrôles
    html.Div([
        # Sélecteur de date
        html.Div([
            html.Label("Sélectionner la période:"),
            dcc.DatePickerRange(
                id='date-range',
                min_date_allowed=df['Date'].min(),
                max_date_allowed=df['Date'].max(),
                start_date=df['Date'].min(),
                end_date=df['Date'].max(),
                style={'marginLeft': '10px'}
            ),
        ], style={'marginBottom': '20px'}),
        
        # Sélecteur du nombre de nœuds top N
        html.Div([
            html.Label("Nombre de nœuds à afficher:"),
            dcc.Slider(
                id='top-n-slider',
                min=5,
                max=50,
                step=5,
                value=10,
                marks={i: str(i) for i in range(5, 51, 5)},
                # style={'width': '500px', 'marginLeft': '10px'}
            ),
        ]),
    ], style={'padding': '20px', 'backgroundColor': '#f8f9fa', 'borderRadius': '5px'}),
    
    # Graphique
    html.Div([
        dcc.Graph(id='congestion-bar-chart')
    ], style={'marginTop': '20px'})
])

@app.callback(
    Output('congestion-bar-chart', 'figure'),
    [Input('date-range', 'start_date'),
     Input('date-range', 'end_date'),
     Input('top-n-slider', 'value')]
)
def update_graph(start_date, end_date, top_n):
    # Filtrer les données par date
    filtered_df = df[
        (df['Date'] >= start_date) &
        (df['Date'] <= end_date)
    ]
    
    # Agréger les données par nœud
    agg_df = filtered_df.groupby('Node')['Congestion'].sum().reset_index()
    
    # Prendre les top N nœuds
    top_nodes = agg_df.nlargest(top_n, 'Congestion')
    
    # Créer le graphique
    fig = px.bar(
        top_nodes,
        x='Node',
        y='Congestion',
        title=f'Top {top_n} Nœuds par Congestion',
        labels={'Node': 'Nœud', 'Congestion': 'Congestion Totale'},
        color='Congestion',
        color_continuous_scale='Viridis'
    )
    
    # Personnaliser le layout
    fig.update_layout(
        xaxis_tickangle=-45,
        bargap=0.2,
        height=600,
        title_x=0.5,  # Centrer le titre
        plot_bgcolor='white',
        paper_bgcolor='white',
        xaxis=dict(
            showgrid=True,
            gridwidth=1,
            gridcolor='LightGray'
        ),
        yaxis=dict(
            showgrid=True,
            gridwidth=1,
            gridcolor='LightGray'
        )
    )
    
    return fig

if __name__ == '__main__':
    app.run(debug=True, port=6050)